In [1]:
import sys
sys.path.insert(1, './src')
sys.path.insert(1, './tokens')
sys.path.insert(1, './data')
sys.path.insert(1, './config')

import pandas as pd
import numpy as np

from src.data_set import lee_limpia_3csv2df

In [2]:
df = lee_limpia_3csv2df()

In [3]:
#generamos un filtro aleatorio
from random import random

lista_pesos = ['habitaciones', 'banos', 'metros', 'fecha_construccion', 'situacion',
       'planta', 'precio_k', 'gastos_comunitarios', 'geo', 'estado', 'NS',
       'EO', 'portal', 'ascensor', 'parque_infantil', 'terraza', 'trastero',
       'piscina_comunitaria', 'cuarto_de_basura', 'zonas_comunes', 'piscina',
       'garaje', 'tejado', 'calefaccion', 'jardin', 'aire_acondicionado']
diccionario_pesos = {p : round(random(), 2) for p in lista_pesos}


In [4]:
#escogemos la fila i como objetivo que lo hacemos aleatorio para nuestras pruebas
i = round(random()*len(df))
target = df.iloc[[i]]
df.index[i]

'UV/2020/000252'

In [5]:
def haversine_vectorize(d, t):
    t = t.loc[t.index.repeat(len(d))]
    indice = d.index
    t.set_index(indice, inplace=True)

    lon1 = d.longitud
    lat1 = d.latitud

    lon2 = t.longitud
    lat2 = t.latitud
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    
    km = 6371* c
    return km


In [6]:
def similitud_numericas(df,taget):
    var_numericas = ['habitaciones', 'banos', 'metros', 'fecha_construccion', 'situacion', 'planta', 'precio_k','gastos_comunitarios']
    df_num = df[var_numericas]
    target_num  = target[var_numericas]
    diferencia= df_num.sub(target_num.iloc[0,:]).abs()
    diferencia_normalizada = diferencia/diferencia.sum()
    similitud = 1 - diferencia_normalizada
    return similitud

In [7]:
def similitud_geo(df, taget):
    var_geo = ['latitud','longitud']
    df_geo = df[var_geo]
    target_geo = target[var_geo]
    distancia = haversine_vectorize(df_geo, target_geo)
    distancia_normalizada = distancia / distancia.max()
    similitud = 1 - distancia_normalizada
    return similitud

In [8]:
def similitud_categoricas(df,target):
    var_categoricas=['estado', 'NS','EO', 'portal', 'ascensor', 'parque_infantil', 'terraza','trastero', 'piscina_comunitaria', 'cuarto_de_basura','zonas_comunes','piscina', 'garaje', 'tejado', 'calefaccion', 'jardin','aire_acondicionado']
    df_cat = df[var_categoricas]
    target_cat = target[var_categoricas]
    target_cat = target_cat.loc[target_cat.index.repeat(len(df))]
    indice=df_cat.index
    target_cat.set_index(indice, inplace=True)
    return (df_cat == target_cat).astype(int)

In [9]:
sim_num = similitud_numericas(df,target)
sim_geo = similitud_geo(df, target)
sim_cat = similitud_categoricas(df,target)

In [10]:
sim_num['geo'] = sim_geo
similitud = sim_num.join(sim_cat)

In [11]:
serie_pesos = pd.Series(diccionario_pesos)

In [12]:
serie_pesos = serie_pesos/serie_pesos.sum()

In [13]:
df['similitud']=round(similitud.dot(serie_pesos),3)

In [14]:
df.drop(index=target.index, inplace= True)
df.sort_values(by='similitud', ascending= False)

,estado,habitaciones,banos,metros,fecha_construccion,NS,EO,situacion,portal,ascensor,...,pais,ciudad,distrito,calle,barrio,planta,tipo_vivienda,precio_k,gastos_comunitarios,similitud
uv,,,,,,,,,,,,,,,,,,,,,
UV/2020/000073,1.0,2.0,1.0,52.0,1960.0,N,,3.0,0.0,1.0,...,1,Madrid,Ciudad Lineal,Calle Betancunia,,5,PISO,160,0.0,0.999
UV/2020/000240,1.0,2.0,1.0,58.0,1965.0,N,,3.0,0.0,1.0,...,1,Madrid,Fuencarral,Avenida Monforte de Lemos,,7,PISO,250,0.0,0.999
UV/2020/000020,1.0,2.0,1.0,84.0,1984.0,N,,3.0,0.0,1.0,...,1,Móstoles,Este,Avenida Carlos V,,3,PISO,175,0.0,0.999
UV/2020/000257,1.0,4.0,1.0,89.0,1936.0,N,,3.0,0.0,1.0,...,1,Barcelona,Sants-Montjuïc,Gran Vía de les Corts Catalanes,,1,0,0,0.0,0.998
UV/2020/000147,1.0,3.0,2.0,133.0,1949.0,N,,3.0,0.0,1.0,...,1,Madrid,Chamberí,Calle Raimundo Lulio,,1,PISO,780,0.0,0.998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UV/2021/000352,1.0,4.0,3.0,174.0,2015.0,S,O,3.0,1.0,1.0,...,1,Madrid,Moncloa,Calle del Doctor Juan José López Ibor,,2,PISO,700,0.0,0.760
UV/2021/000455,1.0,0.0,0.0,40.0,1900.0,N,E,1.0,0.0,0.0,...,1,Madrid,La latina,TEST,Aluche,1,CASA,100,1.0,0.753
UV/2021/000354,1.0,4.0,3.0,174.0,2015.0,S,O,3.0,1.0,1.0,...,1,Madrid,Moncloa,Calle del Doctor Juan José López Ibor,,8,PISO,710,0.0,0.749
